In [ ]:
#Code to Scrap Data from websites   
import json
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date
import time
import spacy 

nlp = spacy.load(r'/Users/vishnupreethamreddydasari/anaconda3/lib/python3.11/site-packages/en_core_web_sm/en_core_web_sm-3.7.0')
f = open('companieslist.json')
# positionname = ['Business Analyst','Data Analyst','Data Science Analyst']
positionname = ['SDE','Software Developer','Front End Developer','Back End Developer','Senior Developer']
data = json.load(f)
f.close() 
for (k, v) in data.items(): 
   driver = webdriver.Safari()  
   driver.get(str(v))  
   time.sleep(2) 
   length = driver.find_elements(By.XPATH, "//div[@data-bind='foreach: opportunities']/div")
   print("Total Number of Positions",len(length))
   time.sleep(4)  
   listofpositions = [] 
   for i in range(0,len(length)): 
      rolename = driver.find_element(By.XPATH, f"//div[@data-bind='foreach: opportunities']//div[@id='Opportunity{i}']//div[@class='row']//div[1]//h3//a")
      x = rolename.text.strip()
      # print(x)
      printlink = driver.find_element(By.XPATH, f"//div[@data-bind='foreach: opportunities']//div[@id='Opportunity{i}']//div[@class='row']//div//h3//a").get_attribute("href")
      # print(printlink)
      name = driver.find_element(By.XPATH, f"//div[@data-bind='foreach: opportunities']//div[@id='Opportunity{i}']//div[@class='row']//div[2]//h3")
      # print(name.text.strip())
      if (name.text.strip() == "Oct 30, 2023"): 
         search_term_tokens = nlp(x)
         for role in positionname:
            role_tokens = nlp(role)
            similarity = search_term_tokens.similarity(role_tokens)

            if similarity > 0.9:
               listofpositions.append(printlink) 
               print(listofpositions)
               break
      else:
         continue
   driver.close()
   for i in range(len(listofpositions)):
      driver = webdriver.Safari()
      driver.get(str(listofpositions[i]))  
      time.sleep(3)
      driver.close()


In [ ]:
import spacy

# Load the spaCy language model
nlp = spacy.load(r'/Users/vishnupreethamreddydasari/anaconda3/lib/python3.11/site-packages/en_core_web_sm/en_core_web_sm-3.7.0')

# Define the list of role names
rolename = ['Business Analyst', 'Data Analyst', 'Data Science Analyst','Software Developer']
positionname = ['SDE','Software Developer','Front End Developer','Back End Developer','Senior Developer']

# Text to search for
search_term = 'Senior Software Developer'

# Tokenize the search term
search_term_tokens = nlp(search_term)

# Iterate through the role names and check for similarity
for role in rolename:
    role_tokens = nlp(role)
    similarity = search_term_tokens.similarity(role_tokens)
    
    if similarity >= 0.85:  # You can adjust the similarity threshold as needed
        print(f"'{search_term}' is similar to '{role}'")


In [ ]:
#Code to send email with links to latest open positions. 
import smtplib
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def emailprocess(listofpositions):
   # Email configuration
   smtp_server = "smtp.gmail.com"
   smtp_port = 465  
   email_sender = "vishnupreethamreddy@gmail.com"
   email_password = ""
   email_receiver = "vdasarireddy2@gmail.com"
   subject = "Open Positions Alert!!"
   message = f"These are following positions opened in lending point. {listofpositions}"

   # Create an instance of MIMEMultipart
   msg = MIMEMultipart()
   msg['From'] = email_sender
   msg['To'] = email_receiver
   msg['Subject'] = subject

   # Attach the message to the email
   msg.attach(MIMEText(message, 'plain'))

   context = ssl.create_default_context()

   # Create an SMTP_SSL connection
   with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as smtp:
      smtp.login(email_sender, email_password)
      smtp.sendmail(email_sender, email_receiver, msg.as_string())

   print("Email sent successfully!")

emailprocess(listofpositions)